In [88]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import random
from collections import Counter

from generator import generateCliffordCircuit
# from execution.transpiler import transpileListToQiskitCircuit

In [3]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [4]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['Z', 'H', 'X', 'Z', 'I'], ['CNOT_C', 'I', 'CNOT_T', 'I', 'I'], ['H', 'X', 'S', 'X', 'H'], ['I', 'I', 'CNOT_C', 'CNOT_T', 'I'], ['H', 'H', 'I', 'H', 'Z']]
     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ Z ├─░───■───░─┤ H ├─░───────░─┤ H ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_1: ┤ H ├─░───┼───░─┤ X ├─░───────░─┤ H ├
     ├───┤ ░ ┌─┴─┐ ░ ├───┤ ░       ░ ├───┤
q_2: ┤ X ├─░─┤ X ├─░─┤ S ├─░───■───░─┤ I ├
     ├───┤ ░ └───┘ ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_3: ┤ Z ├─░───────░─┤ X ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ └───┘ ░ ├───┤
q_4: ┤ I ├─░───────░─┤ H ├─░───────░─┤ Z ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [5]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")


In [6]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('-ZZIXX')

In [7]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [8]:
def G_twirling(circuit):
    '''This function performs G-twirling on a circuit. It randomly chooses a Pauli from {X,Y,Z,I} and prepends it before a layer, then appends P'=CPC^T after the layer.}
    Inputs: circuit - A Qiskit Circuit object.
    Outputs: new_circuit - A Qiskit Circuit object.'''
    num_qubits = circuit.num_qubits
    new_circuit = qk.QuantumCircuit(num_qubits)
    for index, subcircuit in enumerate(split_circuit_by_barrier(circuit)):
        # if len(subcircuit.data) == 1:
        #     operated_qubits = [subcircuit.find_bit(q).index for q in subcircuit.data[0].qubits]
        # else:
        #     operated_qubits = list(range(num_qubits))
        pauli_str = ''.join(random.choice(["X","Y","Z","I"]) for _ in range(num_qubits))
        pauli = qk.quantum_info.Pauli(pauli_str)
        new_circuit = new_circuit.compose(pauli,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(Clifford_Permute(subcircuit,pauli).to_instruction(),qubits = range(num_qubits))
        if index != len(list(split_circuit_by_barrier(circuit)))-1:
            new_circuit.barrier()
    return new_circuit


In [9]:
G_twirling(qiskitCircuit).draw()

┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐     »
q_0: ┤0              ├┤ Z ├┤0        ├─░─┤0              ├──■──»
     │               │├───┤│         │ ░ │               │  │  »
q_1: ┤1              ├┤ H ├┤1        ├─░─┤1              ├──┼──»
     │               │├───┤│         │ ░ │               │┌─┴─┐»
q_2: ┤2 Pauli(XYZYZ) ├┤ X ├┤2 -XYZYZ ├─░─┤2 Pauli(XZXZY) ├┤ X ├»
     │               │├───┤│         │ ░ │               │└───┘»
q_3: ┤3              ├┤ Z ├┤3        ├─░─┤3              ├─────»
     │               │├───┤│         │ ░ │               │     »
q_4: ┤4              ├┤ I ├┤4        ├─░─┤4              ├─────»
     └───────────────┘└───┘└─────────┘ ░ └───────────────┘     »
«     ┌───────────────┐ ░ ┌───────────────┐┌───┐┌───────────────┐ ░ »
«q_0: ┤0              ├─░─┤0              ├┤ H ├┤0              ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_1: ┤1              ├─░─┤1              ├┤ X ├┤1              ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_2: ┤2 Pauli(XZIZY) ├─░─┤2 Pauli(IZIYZ) ├┤ S ├┤2 Pauli(IZIYX) ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_3: ┤3              ├─░─┤3              ├┤ X ├┤3              ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_4: ┤4              ├─░─┤4              ├┤ H ├┤4              ├─░─»
«     └───────────────┘ ░ └───────────────┘└───┘└───────────────┘ ░ »
«     ┌───────────────┐     ┌───────────────┐ ░ ┌───────────────┐┌───┐»
«q_0: ┤0              ├─────┤0              ├─░─┤0              ├┤ H ├»
«     │               │     │               │ ░ │               │├───┤»
«q_1: ┤1              ├─────┤1              ├─░─┤1              ├┤ H ├»
«     │               │     │               │ ░ │               │├───┤»
«q_2: ┤2 Pauli(ZIXXZ) ├──■──┤2 Pauli(ZXXXZ) ├─░─┤2 Pauli(YZXZX) ├┤ I ├»
«     │               │┌─┴─┐│               │ ░ │               │├───┤»
«q_3: ┤3              ├┤ X ├┤3              ├─░─┤3              ├┤ H ├»
«     │               │└───┘│               │ ░ │               │├───┤»
«q_4: ┤4              ├─────┤4              ├─░─┤4              ├┤ Z ├»
«     └───────────────┘     └───────────────┘ ░ └───────────────┘└───┘»
«     ┌─────────┐
«q_0: ┤0        ├
«     │         │
«q_1: ┤1        ├
«     │         │
«q_2: ┤2 -YXXXZ ├
«     │         │
«q_3: ┤3        ├
«     │         │
«q_4: ┤4        ├
«     └─────────┘

In [10]:
circuit_ensemble = [G_twirling(qiskitCircuit) for _ in range(10)]
# for circ in circuit_ensemble:
#     print(circ)

In [11]:
def random_choices_with_counts(items, n):
    """
    Randomly choose with replacement n items from a list and return a dictionary
    with the counts of each chosen item.
    
    Parameters:
        items (list): List of items to choose from.
        n (int): Number of items to choose.
    
    Returns:
        dict: Dictionary with the counts of each chosen item.
    """
    chosen_items = random.choices(items, k=n)
    counts = Counter(chosen_items)
    return dict(counts)

In [12]:
def find_circuit_from_name(circuit_list,target_name):
    # Find the circuit with the given name
    found_circuit = None
    for circuit in circuit_list:
        if circuit.name == target_name:
            found_circuit = circuit
            break

    # Check if the circuit was found
    if found_circuit is not None:
        # print(f"Circuit with name '{target_name}' found:")
        # print(found_circuit)
        return found_circuit
    else:
        print(f"No circuit with name '{target_name}' found.")
        return found_circuit

In [13]:
me = qk.QuantumCircuit(5)
me.pauli(pauli_string = "XX",qubits = [1,3])
print(me)

                   
q_0: ──────────────
     ┌────────────┐
q_1: ┤0           ├
     │            │
q_2: ┤  Pauli(XX) ├
     │            │
q_3: ┤1           ├
     └────────────┘
q_4: ──────────────
                   


In [14]:
num_experiments = 1000
freq_dict = random_choices_with_counts([circ.name for circ in circuit_ensemble],num_experiments)
find_circuit_from_name(circuit_ensemble,list(freq_dict.keys())[0])

In [71]:
def prep_circuit(input_pauli):
    '''This function prepares the input state for eigenvalue sampling. It takes in a Pauli operator to be input into the circuit and outputs a circuit that prepares the state |psi+> or |psi-> from the input |0>^n, 
    where |psi+> and |psi-> are the eigenstates of the input Pauli operator.
    Inputs: input_pauli - A Qiskit Pauli object or a string representing a Pauli operator.
    Outputs: prep_circ - A Qiskit Circuit object which will prepare the state |psi+> or |psi-> from |0>^n, where |psi+> and |psi-> are the eigenstates of the input Pauli operator.
    '''
    # mapping for preparation circuit. The idea of the prep circuit is that we append it before the circuit to eigenvalue samp
    # i.e. input the state where all qubits are zero 1000...0> into the prep circuit and then input it's ouput into the circu
    # |0> -> ...
    # H -> |+>
    # XH -> |->
    # HS -> |+i>
    # XHS -> |-i>
    # I -> |0> (no change)
    # X -> |1>
    if type(input_pauli) == str:
        input_pauli_string = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    nontriv_indices = [index for index, char in enumerate(input_pauli_string) if char != 'I'] # finds the indices of the nontrivial Pauli gates

    prep_circ = qk.QuantumCircuit(len(input_pauli_string))
    p_eigenstate = [random.choice(range(2)) for i in nontriv_indices] # randomly chooses |psi+> (0) or |psi-> (1)
    
    for index, qubit in enumerate(nontriv_indices):
        pauli_gate =  input_pauli_string[qubit]
        if pauli_gate == "X": # if the input Pauli is X
            if p_eigenstate[index] == 0: # if the randomly chosen eigenvector is |+>
                prep_circ.h(qubit)
            elif p_eigenstate[index] == 1: # if the randomly chosen eigenvector is |->
                prep_circ.x(qubit)
                prep_circ.h(qubit)
        elif pauli_gate == "Y": # if the input Pauli is Y
            if p_eigenstate[index] == 0: # if the randomly chosen eigenvector is l+i>
                prep_circ.h(qubit)
                prep_circ.s(qubit)
            elif p_eigenstate[index] == 1: # if the randomly chosen eigenvector is |-i>
                prep_circ.x(qubit)
                prep_circ.h(qubit)
                prep_circ.s(qubit)
        elif pauli_gate == "Z": # if the input Pauli is Z
            # we don't need to append anything if the chosen eigenstate is 10>, since the input will already be in that state
            if p_eigenstate[index] == 1: # if the randomly chosen eigenvector is
                prep_circ.x(qubit)

    return prep_circ

In [77]:
def measure_circuit(final_pauli):
    if type(final_pauli) == str:
        final_pauli_string = ''.join(char for char in final_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(final_pauli) == qk.quantum_info.Pauli:
        final_pauli_string = ''.join(char for char in final_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    measurement_circ = qk.QuantumCircuit(len(final_pauli_string),len(final_pauli_string))

    # On the measurement circ, we need to dagger the transforms used to prepare the prep circ to measure in the bases given by P'
    for index, g in enumerate(final_pauli_string):
        if g == "X":
            measurement_circ.h(index) # rotate into X basis
            measurement_circ.measure(index,index) # then measure
        elif g == "Y":
            measurement_circ.s(index).inverse()
            measurement_circ.h(index)# S**-1 H rotates into Y basis
            measurement_circ.measure(index,index)# then measure
        elif g == "Z":
            measurement_circ.measure(index,index)# measure

    return measurement_circ

In [18]:
a = measure_circ("XYIZX")
a.draw()

┌───┐     ┌─┐      
q_0: ┤ H ├─────┤M├──────
     ├───┤┌───┐└╥┘   ┌─┐
q_1: ┤ S ├┤ H ├─╫────┤M├
     └───┘└───┘ ║    └╥┘
q_2: ───────────╫─────╫─
      ┌─┐       ║     ║ 
q_3: ─┤M├───────╫─────╫─
      └╥┘ ┌───┐ ║ ┌─┐ ║ 
q_4: ──╫──┤ H ├─╫─┤M├─╫─
       ║  └───┘ ║ └╥┘ ║ 
c: 5/══╩════════╩══╩══╩═
       3        0  4  1

In [104]:
def est_Lambda(input_pauli,circuit_ensemble,num_experiments):

    if type(input_pauli) == str:
        input_pauli_string = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    freq_dict = random_choices_with_counts([circ.name for circ in circuit_ensemble],num_experiments)
    for circ_name in freq_dict:
        circ = find_circuit_from_name(circuit_ensemble,circ_name)
        num_shots = freq_dict[circ_name]

        prep_circ = prep_circuit(input_pauli_string)

        measure_circ = measure_circuit(Clifford_Permute(circ,qk.quantum_info.Pauli(input_pauli_string)))

        full_circuit = prep_circ.compose(circ).compose(measure_circ)
    
        # Simulate the circuit with the stabilizer simulator
        simulator = Aer.get_backend('aer_simulator', method='stabilizer')
        result = qk.execute(full_circuit, backend = simulator, shots = 200).result()
        counts = result.get_counts()

        

    return counts


In [105]:
est_Lambda("XIZYX",circuit_ensemble,1000)

{'00000': 22,
 '11001': 31,
 '11101': 18,
 '10100': 21,
 '00100': 34,
 '10000': 24,
 '01001': 25,
 '01101': 25}